# MNIST classification with a ConvNet


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras import backend as K
from tensorflow import keras
import numpy as np


## Load MNIST data


In [2]:
# data is split into training and testing sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print(f'Input data shape: {x_train.shape}')
print(f'Output data shape: {y_train.shape}')


Input data shape: (60000, 28, 28)
Output data shape: (60000,)


The data is comprised of 60,000 28x28 pixel images. Depending on the backend of Keras we need to reformat the images to 28x28x1 or 1x28x28


In [3]:
img_rows, img_cols = 28, 28


In [4]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


In [5]:
print(f'New data shape: {input_shape}')


New data shape: (28, 28, 1)


The data type for each pixel is currently an integer between 0 and 255. We need to convert these to single precision floating point numbers and normalize them.


In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255


The classes for each image are for numbers between 0 and 9 (a total of 10 classes) and they need to be one-hot encoded.


In [7]:
num_classes = 10


In [8]:
print(f'Output for the first image: {y_train[0]}')


Output for the first image: 5


In [9]:
# one-hot encoding the labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
print(f'Output for the first image after o.h.e.: {y_train[0]}')


Output for the first image after o.h.e.: [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [10]:
print(f'New shape of the output data: {y_train.shape}')


New shape of the output data: (60000, 10)


Now the data is loaded and ready to go


## Building the model


In [11]:
cnn = models.Sequential([
    layers.Conv2D(filters=25, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

2022-05-16 15:20:18.474277: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn.fit(x_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.2300 - accuracy: 0.9292
Epoch 2/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.0767 - accuracy: 0.9766
Epoch 3/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0557 - accuracy: 0.9829
Epoch 4/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.0437 - accuracy: 0.9865
Epoch 5/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0345 - accuracy: 0.9892
Epoch 6/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0293 - accuracy: 0.9909
Epoch 7/10
1875/1875 [==============================] - 27s 15ms/step - loss: 0.0242 - accuracy: 0.9920
Epoch 8/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.0206 - accuracy: 0.9933
Epoch 9/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0169 - accuracy: 0.9946
Epoch 10/10
1875/1875 [==============================] - 29s 15m

In [13]:
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

cnn.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0565 - accuracy: 0.9855


[0.05647491291165352, 0.9854999780654907]